# Chainer の学習と推論を SageMaker で行う

MNISTデータセットを対象にSageMakerと**独自のChainerのコード**を利用してMLPの学習と推論を行います。学習の方法としては、高性能なトレーニングインスタンスを1つまたは複数立ち上げて学習を行います。


## 目次
1. [準備](#準備)
2. [データの取得とS3へのアップロード](#データの取得とS3へのアップロード)
3. [学習スクリプトの確認](#学習スクリプトの確認)
4. [モデルの学習](#モデルの学習)
5. [モデルの推論を実行](#モデルの推論を実行)
6. [エンドポイントの削除](#エンドポイントの削除)

## データの取得とS3へのアップロード

ここでは、`Chainer` でサポートされている関数を使って、MNIST データをダウンロードします。SageMaker の学習時に利用するデータは、S3 に置く必要があります。ここでは、ローカルにダウンロードした MNIST データを npz 形式で固めてから、SageMaker のラッパー関数を使って S3 にアップロードします。S3に上げる際のファイルの形式は、学習スクリプト内での読み込み処理と一致している限り指定はありません。

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します。当該バケットがない場合には、自動で新しく作成します。`upload_data()` メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することも可能です。

![図1](./images/image1.jpg "図1")

In [ ]:
import chainer
import os
import shutil
import numpy as np

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Download MNIST dataset
print('Downloading Dataset...')
train, test = chainer.datasets.get_mnist()

# Extract data and labels from dataset 
train_images = np.array([data[0] for data in train])
train_labels = np.array([data[1] for data in train])
test_images = np.array([data[0] for data in test])
test_labels = np.array([data[1] for data in test])

# Save the data and labels as .npz into local directories and upload them to S3
print('Uploading Dataset to S3...')
try:
    os.makedirs('/tmp/data/train')
    os.makedirs('/tmp/data/test')

    np.savez('/tmp/data/train/train.npz', images=train_images, labels=train_labels)
    np.savez('/tmp/data/test/test.npz', images=test_images, labels=test_labels)

    train_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'train'),
        key_prefix='notebook/chainer/mnist')
    test_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'test'),
        key_prefix='notebook/chainer/mnist')
finally:
    shutil.rmtree('/tmp/data')
print('Finished')
print('train_input: {}'.format(train_input))
print('test_input: {}'.format(test_input))

## 学習スクリプトの確認

SageMakerで、Chainer、Tensorflowなどのフレームワークを利用して深層学習を行うためには、このnotebook以外に**学習スクリプトを作成する必要があります**。学習スクリプトとはモデルや学習方法を記述した.pyファイルで、このnotebookには`chainer_mnist.py`という学習スクリプトを同じフォルダに用意しています。ノートブックインスタンスでfit関数を呼び出すと、entry_pointに指定したスクリプトを起点に学習が行われます。

chainerを利用する場合は、学習スクリプトの`__main__`関数内にモデルの記述や学習方法を記載すればよく、SageMakerを使う以前のChainerのコードを概ねそのまま利用することができます。また、環境変数経由で入力データの場所や GPU の数などを取得することが可能です。これは `argparse` 経由で `main` 関数内で受け取ることができます。詳細は[こちら](https://sagemaker.readthedocs.io/en/stable/using_chainer.html#preparing-the-chainer-training-script)をご覧ください。

また推論時の処理は、`model_fn` で学習済みモデルをロードする部分だけ記述する必要があります。その他オプションで、前処理、推論処理、後処理部分を `input_fn`、 `predict_fn`、 `output_fn` で書くこともできます。デフォルトでは、`application/x-npy` コンテントタイプで指定される、NumPy 配列を入力として受け取ります。 詳細は[こちら](https://sagemaker.readthedocs.io/en/stable/using_chainer.html#the-sagemaker-chainer-model-server)をご覧ください。

以下のセルを実行して学習スクリプトの中身を表示してみます。すると、`class MLP(chainer.Chain)`といったモデルの定義、`__main__`の中に学習のコードが書かれていることがわかります。また、chainerMNのoptimizerを使用して分散学習を行うようになっています。



In [ ]:
!pygmentize 'chainer_mnist.py'

## モデルの学習

`Estimator` クラスの子クラスの `Chainer` オブジェクトを作成し、`fit()` メソッドで学習ジョブを実行します。 `entry_point` で指定したローカルのスクリプトが、学習用のコンテナ内で実行されます。また合わせて `source_dir` でローカルのディレクトリを指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。`source_dir`の中に`requirements.txt`ファイルを含めることで、学習時にpipで入るパッケージを利用することができます。

単一ノードで学習したい場合は、`train_instance_count=1`として、学習用インスタンスを`instance_type`に指定します。複数ノードによる学習は、`train_instance_count`を1より大きくすることで実行できます。複数ノードの場合には、分散学習となるようにエントリーポイントにChainerMNを利用した実装が必要になります。あとで学習の結果を参照するためにジョブの名前を記録しておきます。

学習が始まると、学習インスタンスから出力されたログがノートブックインスタンスに表示されます。`fit()`メソッドの引数に`wait=False`を指定することで、非同期に学習ジョブを実行することも可能です。

![図2](./images/image2.jpg "図2")

In [ ]:
import subprocess

from sagemaker.chainer.estimator import Chainer

instance_type = 'ml.m4.xlarge'

chainer_estimator = Chainer(entry_point='chainer_mnist.py', role=role,
                            train_instance_count=1, train_instance_type=instance_type,
                            hyperparameters={'epochs': 3, 'batch_size': 128})

chainer_estimator.fit({'train': train_input, 'test': test_input})

# Keep the job name for checking training loss later 
training_job = chainer_estimator.latest_training_job.name

# モデルの推論を実行


推論を行うために学習したモデルをデプロイします。ここでは、学習用インスタンスで学習した結果からデプロイしましょう。`deploy()` メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。

![図3](./images/image3.jpg "図3")

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


デプロイが終わったら実際に手書き文字認識を行ってみましょう。ランダムに5枚選んで推論をしてみます。

In [ ]:
%matplotlib inline
import random

import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(test_images.shape[0] - 1), num_samples)
images, labels = test_images[indices], test_labels[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
predictor.delete_endpoint()